In [ ]:
import pandas as pd
import requests

local = 'ko_KR'
startDate = '20000101'
endDate = '20230331'

# 개별종목 목록
def getStockDataFrame():
    url = f'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

    params = {
        'locale': local,
        'mktsel': 'ALL',
        'typeNo': 0,
        'searchText': '',
        'bld': 'dbms/comm/finder/finder_stkisu',
    }

    res = requests.post(url, data=params)

    df = pd.DataFrame(res.json()['block1'])
    return df

# get excel download code
def getExcelDownloadCode(stockName, fullCode, codeName):
    url = f'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'

    params = {
        'locale': local,
        'tboxisuCd_finder_stkisu0_1': stockName,
        'isuCd': fullCode,
        'isuCd2': '',
        'codeNmisuCd_finder_stkisu0_1': codeName,
        'param1isuCd_finder_stkisu0_1': 'ALL',
        'strtDd': startDate,
        'endDd': endDate,
        'adjStkPrc_check': 'Y',
        'adjStkPrc': 2,
        'share': 1,
        'money': 1,
        'csvxls_isNo': False,
        'name': 'fileDown',
        'url': 'dbms/MDC/STAT/standard/MDCSTAT01701'
    }

    res = requests.post(url, data=params)
    return res.text

# excel download
def downloadExcel(code, stockName):
    url = f'http://data.krx.co.kr/comm/fileDn/download_excel/download.cmd'

    params = {
        'code': code
    }
    
    res = requests.post(url, data=params)
    
    print(stockName + " : " + str(res))
    
    file_name = "stock_data_kr/" + stockName.replace('/', '_')
    if startDate != '':
        file_name += '_' + startDate
    if endDate != '':
        file_name += '_' + endDate
    file_name += '.xlsx'
    
    with open(file_name, 'wb') as f:
        f.write(res.content)

In [ ]:
for i, row in getStockDataFrame().iterrows():
    stockName = row['short_code'] + '/' + row['codeName']
    fullCode = row['full_code']
    codeName = row['codeName']
    
    code = getExcelDownloadCode(stockName, fullCode, codeName)
    downloadExcel(code, stockName)